## Dataset Overview

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

In [3]:
df=pd.read_csv("/content/drive/MyDrive/MiniProject/overview-of-recordings.csv")

In [4]:
df_text = df[['phrase', 'prompt']]
df_text

,phrase,prompt
0,When I remember her I feel down,Emotional pain
1,When I carry heavy things I feel like breaking...,Hair falling out
2,there is too much pain when i move my arm,Heart hurts
3,My son had his lip pierced and it is swollen a...,Infected wound
4,My muscles in my lower back are aching,Infected wound
...,...,...
6656,I feel a burning sensation in my guts about 2 ...,Stomach ache
6657,I have a split on my thumb that will not heal.,Open wound
6658,I feel a lot of pain in the joints.,Joint pain
6659,The area around my heart doesn't feel good.,Heart hurts


In [5]:
# Check the distribution of phrases and prompts
print("Unique phrases:", df['phrase'].nunique())
print("Unique prompts:", df['prompt'].nunique())

Unique phrases: 706
Unique prompts: 25


## Pandas profiling

In [6]:
#!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

In [7]:
from pandas_profiling import ProfileReport
from ydata_profiling import ProfileReport
profile=ProfileReport(df)
profile.to_file("pandas_profile.html")

<ipython-input-7-cd66952a0adb>:1: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport
/usr/local/lib/python3.10/dist-packages/ydata_profiling/profile_report.py:354: UserWarning: Try running command: 'pip install --upgrade Pillow' to avoid ValueError
  warnings.warn(


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## NLP EDA

In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenize text
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Join tokens back into a string
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

In [ ]:
# Apply preprocessing to 'phrase' and 'prompt' columns
df['clean_phrase'] = df['phrase'].apply(preprocess_text)
df['clean_prompt'] = df['prompt'].apply(preprocess_text)

## Applying LSTM on initial preprocessed data

>Import ANN Libraries

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split

In [ ]:
# Example dataset (replace it with your actual dataset)
phrases = df['phrase'].tolist()
prompts = df['prompt'].tolist()

# Tokenize text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(phrases)
X = tokenizer.texts_to_sequences(phrases)

# Padding sequences
max_length = max([len(seq) for seq in X])
print("Max_length=",max_length)
X_padded = pad_sequences(X, maxlen=max_length, padding='post')

# Convert labels to one-hot encoding
label_to_index = {label: i for i, label in enumerate(set(prompts))}
y = [label_to_index[label] for label in prompts]
y_one_hot = np.zeros((len(y), len(label_to_index)))
for i, label_index in enumerate(y):
    y_one_hot[i, label_index] = 1

Max_length= 30


>Training model using LSTM

In [ ]:
from keras.callbacks import EarlyStopping

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_one_hot, test_size=0.2, random_state=42)

# Define model architecture
embedding_dim = 100
vocab_size = len(tokenizer.word_index) + 1

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
model.add(LSTM(units=128))
model.add(Dense(units=len(label_to_index), activation='softmax'))

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train model with early stopping
history = model.fit(X_train, y_train, epochs=25, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


Epoch 1/25
134/134 [==============================] - 9s 54ms/step - loss: 3.0525 - accuracy: 0.0777 - val_loss: 2.6096 - val_accuracy: 0.1351
Epoch 2/25
134/134 [==============================] - 2s 18ms/step - loss: 2.2839 - accuracy: 0.1945 - val_loss: 1.9654 - val_accuracy: 0.2674
Epoch 3/25
134/134 [==============================] - 2s 12ms/step - loss: 1.6730 - accuracy: 0.3510 - val_loss: 1.5928 - val_accuracy: 0.3987
Epoch 4/25
134/134 [==============================] - 2s 12ms/step - loss: 1.2498 - accuracy: 0.5042 - val_loss: 1.0560 - val_accuracy: 0.6238
Epoch 5/25
134/134 [==============================] - 1s 10ms/step - loss: 0.8843 - accuracy: 0.6727 - val_loss: 0.8451 - val_accuracy: 0.7411
Epoch 6/25
134/134 [==============================] - 2s 11ms/step - loss: 0.5726 - accuracy: 0.8229 - val_loss: 0.5670 - val_accuracy: 0.8180
Epoch 7/25
134/134 [==============================] - 2s 18ms/step - loss: 0.3612 - accuracy: 0.9038 - val_loss: 0.2513 - val_accuracy: 0.9503